# 巧算题目的构造思路--单级运算 （加减法 only）
1. 只考虑整数（允许负数）
2. 从整数结果出发，随机选择op，基于op的逆向来随机生成操作数。res = num1 op num2. 例如，随机生成res=200，随机选择 op + 或者 -，随机生成另外一个数num1，然后根据+或者-，计算num2
3. 
4.
5. 例如这道题： 123 + 78 - 23 + 122 = (122 +78) + (123-23) = 200 +100 = 300，思路
   1). 先随机生成大整数 300, 随机选取得到200，



# 巧算的题目，多级运算构造思路 （加减法 only）
从最终结果出发，按照级别